<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/Assignment4/i_Hyperparameter_search_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objective : To find if hyperparameter search of keras tuner improves accuracy of multi class classification.

The dataset used is the CIFAR-10 dataset, which consists of 50,000 training images and 10,000 testing images of different classes (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck). The goal is to train a model that can accurately classify images into one of these 10 categories. 

In [ ]:
!pip install tensorflow-gpu
!pip install keras-tuner

In [4]:
# imports
import tensorflow as tf
from tensorflow import keras
import keras_tuner
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras.datasets import cifar10


<ipython-input-4-a31f5f22114b>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [49]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Select a subset of the data for training and testing
train_subset = 15000
test_subset = 500
x_train = x_train[:train_subset]
y_train = y_train[:train_subset]
x_test = x_test[:test_subset]
y_test = y_test[:test_subset]

# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoding
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


define a simple CNN model with 3 convolutional layers and 2 fully connected layers



In [50]:
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


train the model for 20 epochs and evaluate its accuracy on the test set

In [51]:
# Create and train the model
model = create_model()
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/20
469/469 [==============================] - 5s 8ms/step - loss: 1.7645 - accuracy: 0.3436 - val_loss: 1.5086 - val_accuracy: 0.4540
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 1.3980 - accuracy: 0.4895 - val_loss: 1.3290 - val_accuracy: 0.5160
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 1.2403 - accuracy: 0.5566 - val_loss: 1.2979 - val_accuracy: 0.5140
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 1.1118 - accuracy: 0.6078 - val_loss: 1.2018 - val_accuracy: 0.5460
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.9989 - accuracy: 0.6481 - val_loss: 1.1446 - val_accuracy: 0.5960
Epoch 6/20
469/469 [==============================] - 3s 7ms/step - loss: 0.9022 - accuracy: 0.6811 - val_loss: 1.0898 - val_accuracy: 0.6000
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.8092 - accuracy: 0.7135 - val_loss: 1.1085 - val_accuracy: 0.6040
Epoch 

Hyperparameter tuning with Keras Tuner:
We will use the RandomSearch tuner to randomly sample values for the hyperparameters, and we will search over the learning rate, number of filters in the convolutional layers, and number of neurons in the fully connected layers.

In [57]:
from kerastuner.tuners import RandomSearch

# Define the hyperparameter search space
def create_model(hp):
    model = keras.Sequential()
    model.add(layers.Conv2D(hp.Int('filters1', 32, 96, step=32), (3,3), activation='relu', input_shape=x_train.shape[1:]))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(hp.Int('filters2', 32, 96, step=32), (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(hp.Int('filters3', 32, 96, step=32), (3,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('units1', 64, 256, step=32), activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [58]:
# Define the tuner
tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='keras_tuner',
    project_name='cifar10'
)

In [59]:
# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=30, validation_data=(x_test, y_test))

In [60]:
# Print the best hyperparameters
best_hyperparams = tuner.get_best_hyperparameters()[0]
print('Best hyperparameters:')
print(f'learning_rate = {best_hyperparams.get("learning_rate")}')
print(f'filters1 = {best_hyperparams.get("filters1")}')
print(f'filters2 = {best_hyperparams.get("filters2")}')
print(f'filters3 = {best_hyperparams.get("filters3")}')
print(f'units1 = {best_hyperparams.get("units1")}')

Best hyperparameters:
learning_rate = 0.001
filters1 = 32
filters2 = 64
filters3 = 128
units1 = 192


retrieve the best hyperparameters and create a new model with those values

In [61]:
# Create a new model with the best hyperparameters
model = tuner.hypermodel.build(best_hyperparams)
model.compile(optimizer=keras.optimizers.Adam(best_hyperparams.get('learning_rate')),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for 20 epochs with the best hyperparameters
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test set with the best hyperparameters
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy with hyperparameter tuning:', test_acc)

Epoch 1/10
469/469 [==============================] - 5s 7ms/step - loss: 1.7371 - accuracy: 0.3524 - val_loss: 1.4866 - val_accuracy: 0.4440
Epoch 2/10
469/469 [==============================] - 3s 6ms/step - loss: 1.3865 - accuracy: 0.4971 - val_loss: 1.2507 - val_accuracy: 0.5540
Epoch 3/10
469/469 [==============================] - 3s 6ms/step - loss: 1.2149 - accuracy: 0.5682 - val_loss: 1.1859 - val_accuracy: 0.5600
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 1.0940 - accuracy: 0.6182 - val_loss: 1.2590 - val_accuracy: 0.5680
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 0.9833 - accuracy: 0.6533 - val_loss: 1.1247 - val_accuracy: 0.6100
Epoch 6/10
469/469 [==============================] - 3s 6ms/step - loss: 0.8673 - accuracy: 0.6961 - val_loss: 1.0656 - val_accuracy: 0.6220
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.7811 - accuracy: 0.7279 - val_loss: 1.0705 - val_accuracy: 0.6680
Epoch 

In [62]:
print('Test accuracy with hyperparameter tuning:', test_acc)

Test accuracy with hyperparameter tuning: 0.6679999828338623


After hyper parameter tuning, test accuracy increases to 66% and loss decreases considerably to 1.13 while test accuracy without hyper parameter training was 61% and loss was 2.28